In [ ]:
try:
    from openmdao.utils.notebook_utils import notebook_mode
except ImportError:
    !python -m pip install openmdao[notebooks]

# BroydenSolver

BroydenSolver is a quasi-Newton solver that implements Broyden's second method to solve for values of the model's states that drive their residuals to zero. It does so by maintaining an approximation to the inverse of the Jacobian of the model or a subset of the model. In some cases this can be more efficient than NewtonSolver because updating the approximated inverse Jacobian is cheaper than solving the linear system. It may take more iterations because the search direction depends on an approximation, but the iterations take fewer operations.

The BroydenSolver has two different modes of operation. It can operate on the entire model and solve for every state in the containing system and all subsystems. Alternatively, it can operate on a subset of the model and only solve for a list of states that you provide. The advantage of full-model mode is that you don't have to worry about forgetting a state, particularly in large models where you might not be familiar with every component or variable. The disadvantage is that you are computing the inverse of a larger matrix every time you recalculate the inverse jacobian, though ideally you are not recomputing this very often. Operating on a subset of states is more efficient in both the linear solve and the Broyden update, but you do run the risk of missing a state. The BroydenSolver will print a warning if it finds any states in the model that aren't covered by a solver.


## BroydenSolver Options

In [ ]:
import openmdao.api as om
om.show_options_table("openmdao.solvers.nonlinear.broyden.BroydenSolver")

The BroydenSolver also contains a slot for a linear solver and a slot for a linesearch. See the [linesearch section](linesearch-section) for more about these.

## BroydenSolver Constructor

The call signature for the `BroydenSolver` constructor is:

```{eval-rst}
    .. automethod:: openmdao.solvers.nonlinear.broyden.BroydenSolver.__init__
        :noindex:
```

## BroydenSolver on a Full Model

Here we show an example that uses the [electrical circuit model](../../../advanced_user_guide/models_implicit_components/models_with_solvers_implicit.ipynb) from the
advanced guide. We have replaced the `NewtonSolver` with a `BroydenSolver`, and set the maximum number of iterations to 20. We also assign a `DirectSolver` into the "linear_solver" slot on the `BroydenSolver`.  This is the linear solver that will be used to assemble the Jacobian and compute its inverse. Since we don't specify any states in the `state_vars` option, the BroydenSolver operates on the entire model. If you don't specify a linear_solver here, then the BroydenSolver will use the one from the system.

```{Note}
In this mode, only the `DirectSolver` can be used as the linear_solver.
```

Depending on the values of some of the other options such as "converge_limit", "diverge_limit", and "max_converge_failures", the Jacobian might be recalculated if convergence stalls, though this doesn't happen in the electrical circuit example.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src21", get_code("openmdao.test_suite.scripts.circuit_analysis.Circuit"), display=False)

:::{Admonition} `Circuit` class definition 
:class: dropdown

{glue:}`code_src21`
:::

In [ ]:
import openmdao.api as om
from openmdao.test_suite.scripts.circuit_analysis import Circuit

p = om.Problem()
model = p.model

model.add_subsystem('circuit', Circuit(), promotes_inputs=[('Vg', 'V'), ('I_in', 'I')])
model.set_input_defaults('V', 0., units='V')
model.set_input_defaults('I', 0.1, units='A')

p.setup()

# Replace existing solver with BroydenSolver
model.circuit.nonlinear_solver = om.BroydenSolver()
model.circuit.nonlinear_solver.options['maxiter'] = 20
model.circuit.nonlinear_solver.linear_solver = om.DirectSolver()

# set some initial guesses
p.set_val('circuit.n1.V', 10.)
p.set_val('circuit.n2.V', 1.)

p.set_solver_print(level=2)
p.run_model()

In [ ]:
print(p.get_val('circuit.n1.V'))
print(p.get_val('circuit.n2.V'))

# sanity check: should sum to .1 Amps
print(p.get_val('circuit.R1.I') + p.get_val('circuit.D1.I'))

In [ ]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p.get_val('circuit.n1.V'), 9.90804735, 1e-5)
assert_near_equal(p.get_val('circuit.n2.V'), 0.71278226, 1e-5)

# sanity check: should sum to .1 Amps
assert_near_equal(p.get_val('circuit.R1.I') + p.get_val('circuit.D1.I'), .1, 1e-6)

## BroydenSolver on a Subset of States

The `BroydenSolver` can also be used to solve for specific states. Here we consider the same circuit example, but instead we specify the two voltages n1.V' and 'n2.V' as our "state_vars".  In this mode, we aren't limited to just using the `DirectSolver`, and in this example we choose `LinearBlockGS` instead.

In [ ]:
from openmdao.test_suite.scripts.circuit_analysis import Circuit

p = om.Problem()
model = p.model

model.add_subsystem('circuit', Circuit(), promotes_inputs=[('Vg', 'V'), ('I_in', 'I')])
model.set_input_defaults('V', 0., units='V')
model.set_input_defaults('I', 0.1, units='A')

p.setup()

# Replace existing solver with om.BroydenSolver
model.circuit.nonlinear_solver = om.BroydenSolver()
model.circuit.nonlinear_solver.options['maxiter'] = 20

# Specify states for Broyden to solve
model.circuit.nonlinear_solver.options['state_vars'] = ['n1.V', 'n2.V']

model.nonlinear_solver.linear_solver = om.LinearBlockGS()

# set some initial guesses
p.set_val('circuit.n1.V', 10.)
p.set_val('circuit.n2.V', 1.)

p.set_solver_print(level=2)
p.run_model()

In [ ]:
print(p.get_val('circuit.n1.V'))
print(p.get_val('circuit.n2.V'))

In [ ]:
# sanity check: should sum to .1 Amps
print(p.get_val('circuit.R1.I') + p.get_val('circuit.D1.I'))

In [ ]:
assert_near_equal(p.get_val('circuit.n1.V'), 9.90804735, 1e-5)
assert_near_equal(p.get_val('circuit.n2.V'), 0.71278226, 1e-5)

# sanity check: should sum to .1 Amps
assert_near_equal(p.get_val('circuit.R1.I') + p.get_val('circuit.D1.I'), .1, 1e-6)

## BroydenSolver for Models Without Derivatives

The `BroydenSolver` can be used for models where you don't have any partial derivatives defined, and don't wish to use finite difference to calculate them. This behavior is activated by setting the "compute_jacobian" option to False. Instead of calculating an initial Jacobian, we start with an estimate that is just the identity matrix scaled by a tunable parameter in the options called "alpha". As the `BroydenSolver` iterates, this estimate of the Jacobian is improved, and for some problems, a solution can be reached that satisfies the residual equations.

In this example, we solve for the coupling variable in a version of the Sellar model that severs the cycle
and expresses the difference across the broken cycle as an implicit state, which the `BroydenSolver` will
solve.

In [ ]:
from openmdao.utils.notebook_utils import get_code
from myst_nb import glue
glue("code_src22", get_code("openmdao.test_suite.components.sellar.SellarStateConnection"), display=False)

:::{Admonition} `SellarStateConnection` class definition 
:class: dropdown

{glue:}`code_src22`
:::

In [ ]:
from openmdao.test_suite.components.sellar import SellarStateConnection

prob = om.Problem()
model = prob.model = SellarStateConnection(nonlinear_solver=om.BroydenSolver(),
                                           linear_solver=om.LinearRunOnce())

prob.setup()

model.nonlinear_solver.options['state_vars'] = ['state_eq.y2_command']
model.nonlinear_solver.options['compute_jacobian'] = False

prob.set_solver_print(level=2)
prob.run_model()

In [ ]:
print(prob['y1'])
print(prob['state_eq.y2_command'])

In [ ]:
assert_near_equal(prob['y1'], 25.58830273, .00001)
assert_near_equal(prob['state_eq.y2_command'], 12.05848819, .00001)

## BroydenSolver Option Examples

There are a few additional options that give you more control over when and how often the Jacobian is recomputed.
The "diverge_limit" option allows you to define a limit to the ratio of current residual and the previous iteration's residual above which the solution is considered to be diverging. If this limit is exceeded, then the Jacobian is always recomputed on the next iteration. There is also a "converge_limit" that allows you similarly define a limit above which the solution is considered to be non-converging. When this limit is exceeded, the Jacobian is not immediately recomputed until the limit has been exceeded a number of consecutive times as defined by the "max_converge_failures" option. The default value for "max_converge_failures" is 3, and the default "converge_limit" is 1.0. Exploring these options can help you solve more quickly (or in some cases solve at all) some tougher problems.

Here, we take the same circuit example from above and specify a much lower "converge_limit" and "max_converge_failures" to force recomputation of the Jacobian much more frequently. This results in a quicker convergence in terms of the number of iterations, though keep in mind that solving for the derivatives adds computational cost.

In [ ]:
from openmdao.test_suite.scripts.circuit_analysis import Circuit

p = om.Problem()
model = p.model

model.add_subsystem('circuit', Circuit(), promotes_inputs=[('Vg', 'V'), ('I_in', 'I')])
model.set_input_defaults('V', 0., units='V')
model.set_input_defaults('I', 0.1, units='A')

p.setup()

# Replace existing solver with BroydenSolver
model.circuit.nonlinear_solver = om.BroydenSolver()
model.circuit.nonlinear_solver.options['maxiter'] = 20
model.circuit.nonlinear_solver.options['converge_limit'] = 0.1
model.circuit.nonlinear_solver.options['max_converge_failures'] = 1

# Specify states for Broyden to solve
model.circuit.nonlinear_solver.options['state_vars'] = ['n1.V', 'n2.V']

# set some initial guesses
p.set_val('circuit.n1.V', 10.)
p.set_val('circuit.n2.V', 1.)

p.set_solver_print(level=2)
p.run_model()

In [ ]:
print(p.get_val('circuit.n1.V'))
print(p.get_val('circuit.n2.V'))

In [ ]:
# sanity check: should sum to .1 Amps
print(p.get_val('circuit.R1.I') + p.get_val('circuit.D1.I'))

In [ ]:
assert_near_equal(p.get_val('circuit.n1.V'), 9.90804735, 1e-5)
assert_near_equal(p.get_val('circuit.n2.V'), 0.71278226, 1e-5)

# sanity check: should sum to .1 Amps
assert_near_equal(p.get_val('circuit.R1.I') + p.get_val('circuit.D1.I'), .1, 1e-6)

**stall_limit, stall_tol, and stall_tol_type **

  In some cases, nonlinear solvers can stall out where the norm of the residual stops changing at all. This
  can happen for a couple of reasons. You can hit numerical noise problems and just be wandering around in
  a circle, or you can get stuck on a bound and the line search just keeps running into the same spot no
  matter what. Either way, if you have say 100 max iterations and you stall at 15 ... you waste a lot of
  compute time. To remedy this, you can turn on stall detection in all nonlinear solvers by setting the
  "stall_limit" option to a number greater than zero.

  In this example, we set stall_limit to 3. While the solver iterates, it will compare the value of the
  residual norm to the value computed in the previous iteration.  If the value matches for three iterations
  in a row, then iteration will terminate due to detection of a stall. If "err_on_non_converge" is set
  to True, then an ``AnalysisError`` will be raised just as if we had reached the iteration count limit.

  We also set the `stall_tol` to 1e-6, which is the threshold below which a change in the relative residual
  norm is considered to be unchanged. The option also exists to use the absolute residual norm by setting
  `stall_tol_type` to `'abs'`.

In [ ]:
prob = om.Problem()

prob.model.add_subsystem('comp', om.ExecComp('y=3*x+1'), promotes=['*'])

balance = prob.model.add_subsystem('balance', om.BalanceComp(),
                                   promotes=['*'])
balance.add_balance('x', lower=-.1, upper=10, rhs_val=0, lhs_name='y')

nl_solver = prob.model.nonlinear_solver = om.BroydenSolver()
nl_solver.options['stall_limit'] = 3
nl_solver.options['stall_tol'] = 1e-8
nl_solver.options['maxiter'] = 100

prob.model.linear_solver = om.DirectSolver()

prob.setup()
prob.set_solver_print()

prob.run_model()